<a href="https://colab.research.google.com/github/elymsyr/MLProject/blob/main/RunColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Python Version and Packages

In [ ]:
!rm -r /content/sample_data

In [ ]:
!python --version

Python 3.10.12


Downgrade the python version to v3.9. Install necessary packages. You can find it on [github.com/elymsyr/MLProject](https://github.com/elymsyr/MLProject) (/Docs/requirements.txt)

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.9

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1

#check python version
!python --version

# install pip for new python
!sudo apt-get install python3.9-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

# install colab's dependencies
!python -m pip install ipython tensorflow==2.13.0 tensorboard ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.9/dist-packages/google

# Change between versions
# !sudo update-alternatives --config python3 <RowNumber>

!python --version

In [ ]:
!python -m pip install --upgrade pip
!pip install --upgrade setuptools pip wheel
!pip install mlagents==0.30.0
!pip install mlagents-envs==0.30.0

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# !pip3 install torch torchvision torchaudio

In [ ]:
!pip install nvidia-pyindex
# !pip install --upgrade nvidia-tensorrt

In [ ]:
!pip install protobuf==3.20.3 install numpy==1.23.5 onnx==1.15.0

# Check Environment

## Test Tensorflow-Pytorch GPU

In [ ]:
import torch
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
# Check if GPU is available
!nvcc --version
!nvidia-smi
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        print(f"Number of available GPU devices: {num_gpus}")
        # Iterate over available GPUs and print their index and name
        for gpu_index in range(num_gpus):
            gpu_name = torch.cuda.get_device_name(gpu_index)
            print(f"GPU {gpu_index}: {gpu_name}")
    else:
        print("No GPU devices available.")
else:
    print("GPU is not available.")

## Test MLAgents

In [ ]:
!mlagents-learn -h

# Get Build

In [15]:
!git clone --branch main https://github.com/elymsyr/MLProject.git

Cloning into 'MLProject'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1191 (delta 15), reused 12 (delta 12), pack-reused 1170
Receiving objects: 100% (1191/1191), 435.91 MiB | 26.96 MiB/s, done.
Resolving deltas: 100% (632/632), done.
Updating files: 100% (527/527), done.


In [17]:
# Set the permissions
env = 'b061'
exe = 'b061'

!chmod -R 755 /content/MLProject/Jupyter/{env}/{exe}.x86_64
!chmod -R 755 /content/MLProject/Jupyter/{env}/UnityPlayer.so
!ls -l /content/MLProject/Jupyter/{env}

total 112816
drwxr-xr-x 3 root root     4096 Mar 16 13:10 b061_BurstDebugInformation_DoNotShip
drwxr-xr-x 6 root root     4096 Mar 16 13:10 b061_Data
-rw-r--r-- 1 root root    16208 Mar 16 13:10 b061_s.debug
-rwxr-xr-x 1 root root    15096 Mar 16 13:10 b061.x86_64
-rw-r--r-- 1 root root 67164248 Mar 16 13:10 UnityPlayer_s.debug
-rwxr-xr-x 1 root root 48314296 Mar 16 13:10 UnityPlayer.so


## Create config.yaml

In [22]:
num_envs = 10
env = 'b061'
exe = 'b061'
env_path = f'/content/MLProject/Jupyter/{env}/{exe}.x86_64'
hidden_units = 256
num_layers = 3
learning_rate = 0.0003
beta = 0.005
epsilon = 0.2
lambd = 0.95
num_epoch = 3

from jinja2 import Template

# Define the YAML template
yaml_template = """
default_settings: null
behaviors:
  AgentBehavior:
    trainer_type: ppo
    hyperparameters:
      batch_size: 1024
      buffer_size: 10240
      learning_rate: 0.0003
      beta: {{ beta }}
      epsilon: {{ epsilon }}
      lambd: {{ lambd }}
      num_epoch: {{ num_epoch }}
      shared_critic: false
      learning_rate_schedule: linear
      beta_schedule: linear
      epsilon_schedule: linear
    network_settings:
      normalize: false
      hidden_units: {{ hidden_units }}
      num_layers: {{ num_layers }}
      vis_encode_type: simple
      memory: null
      goal_conditioning_type: hyper
      deterministic: false
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
        network_settings:
          normalize: false
          hidden_units: {{ hidden_units }}
          num_layers: {{ num_layers }}
          vis_encode_type: simple
          memory: null
          goal_conditioning_type: hyper
          deterministic: false
    init_path: null
    keep_checkpoints: 5
    checkpoint_interval: 500000
    max_steps: 10000000
    time_horizon: 64
    summary_freq: 10000
    threaded: false
    self_play: null
    behavioral_cloning: null
env_settings:
  env_path: {{ env_path }}
  env_args: null
  base_port: 5004
  num_envs: {{ num_envs }}
  num_areas: 1
  seed: -1
  max_lifetime_restarts: 10
  restarts_rate_limit_n: 1
  restarts_rate_limit_period_s: 60
engine_settings:
  width: 84
  height: 84
  quality_level: 5
  time_scale: 20.0
  target_frame_rate: -1
  capture_frame_rate: 60
  no_graphics: true
environment_parameters: null
checkpoint_settings:
  run_id: ColabTest
  initialize_from:
  load_model: false
  resume: false
  force: false
  train_model: false
  inference: false
  results_dir: results
torch_settings:
  device: cuda
debug: false
"""

# Render the YAML template with variables
yaml_rendered = Template(yaml_template).render(
    beta=beta,
    epsilon=epsilon,
    lambd=lambd,
    num_epoch=num_epoch,
    hidden_units=hidden_units,
    num_layers=num_layers,
    env_path=env_path,
    num_envs=num_envs
)

# Write the rendered YAML to a file
with open(f"config.yaml", "w+") as yaml_file:
    yaml_file.write(yaml_rendered)

# Tenserboard and Train Process

## Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results
%reload_ext tensorboard

## Delete previous trains

In [ ]:
!rm -r /content/results # if necessary

## Train

In [ ]:
!mlagents-learn /content/config.yaml --env /content/MLProject/Jupyter/b061/b061.x86_64 --resume

## Create and Download Zip Results

In [2]:
!zip -r /content/files.zip /content/results

from google.colab import files
files.download('/content/files.zip')